<a href="https://colab.research.google.com/github/dakilaledesma/FSUS_Explanations/blob/main/notebooks/train/CNL_Q600G4F15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
! unzip -q drive/MyDrive/UNC/S2022P/GBIF_Q_fourth15.zip -d /content/temp
! mv /content/temp/content/dataset/train /content/train
! rm -rf /content/temp

CPU times: user 673 ms, sys: 127 ms, total: 800 ms
Wall time: 1min 30s


In [2]:
! git clone https://github.com/rwightman/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 10858, done.
remote: Counting objects: 100% (385/385), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 10858 (delta 226), reused 277 (delta 183), pack-reused 10473
Receiving objects: 100% (10858/10858), 20.46 MiB | 14.87 MiB/s, done.
Resolving deltas: 100% (7938/7938), done.


In [3]:
import fileinput
import sys

def replacement(file, previousw, nextw):
   for line in fileinput.input(file, inplace=1):
       line = line.replace(previousw, nextw)
       sys.stdout.write(line)

file = "/content/pytorch-image-models/timm/utils/checkpoint_saver.py"
replacement(file, "if os.path.exists(last_save_path):", "# if os.path.exists(last_save_path):")
replacement(file, "os.unlink(last_save_path)  # required for Windows support.", "# os.unlink(last_save_path)  # required for Windows support.")
replacement(file, "os.link(last_save_path, save_path)", "# os.link(last_save_path, save_path)")
replacement(file, "os.unlink(best_save_path)", "# os.unlink(best_save_path)")
replacement(file, "os.link(last_save_path, best_save_path)", "# os.link(last_save_path, best_save_path)")
replacement(file, "if os.path.exists(best_save_path):", "# if os.path.exists(best_save_path):")


In [4]:
from glob import glob

print(len(list(glob("train/*"))))

57


In [ ]:
! python -u -m torch.distributed.launch --nproc_per_node=1 --nnodes=1 --node_rank=0 ./pytorch-image-models/train.py train --model convnext_large_384_in22ft1k --opt adabelief --lr 0.00005 --epochs 80 --aa augmix-m5-w4 --aug-splits 2 --jsd-loss --decay-epochs 3 --cooldown-epochs 0 --weight-decay 1e-4 --sched cosine -b 4 --input-size 3 600 600 --num-classes=60 --vflip 0.5 --hflip 0.5 --amp --pretrained --output drive/MyDrive/UNC/S2022P/output/ 

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
Training with a single process on 1 GPUs.
Loading pretrained weights from url (https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_1k_384.pth)
Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_1k_384.pth" to /root/.cache/torch/hub/checkpoints/convnext_large_22k_1k_384.pth
